In [1]:
import os
import gc
import time
import threading
from datetime import datetime
import urllib3
from itertools import permutations
import pandas as pd
import numpy as np
from numpy import vectorize
from numpy.random import SeedSequence

EXECUTION_NAME = datetime.now().replace(second=0, microsecond=0).isoformat()
PREPROCESSING_EXECUTION_NAME = "2024-05-12T20:24:00"
OUTPUT_PATH = '../output/'

EXECUTION_DURATION_S = 14*24*60*60; NS_TO_S = 1000000000

with open(OUTPUT_PATH+PREPROCESSING_EXECUTION_NAME+'-SEED.csv', 'r') as f:
    seedz = np.vectorize(int)(f.read().split(','))
assert(seedz is not None)    

RANDOM_GEN = np.random.default_rng(seed=SeedSequence(entropy=list(map(abs,seedz))))

In [2]:
############################################ PHASE 2
DF = pd.read_csv(OUTPUT_PATH+PREPROCESSING_EXECUTION_NAME+'-EVENTS.csv.gz',index_col='id')

In [3]:
DF["dur_ns"].describe(percentiles=[.25,.5,.75,.9,.95,.99,.999,.9999])

count     3.961902e+06
mean      3.347944e+09
std       1.850717e+10
min       0.000000e+00
25%       1.000000e+06
50%       3.100000e+07
75%       3.720000e+08
90%       1.620000e+09
95%       1.441600e+10
99%       7.219999e+10
99.9%     2.536556e+11
99.99%    4.660390e+11
max       5.786200e+11
Name: dur_ns, dtype: float64

In [ ]:
# TODO class for measure

In [16]:
#max_func_per_app = max(len(DF[DF['app_id'] == x]["func_id"].unique()) for x in DF["app_id"].unique())
CPU_TIME_STATS = ["cpu", 
              "user", 
              "system", 
              "idle",
              "nice", 
              "iowait", 
              "irq", 
              "softirq",
              "steal", 
              "guest", 
              "guestNice"]
PROC_STATS = ["used", "available"]
MEM_STATS = ["used", "available"]
        
N_VCPUS = os.cpu_count()


EVENT_FEAT = 
['client_init_ts_ns', # ts
'app',
'func',
'rid', # id
'begin',
'duration',
'end',
'arrival',
# parameter estimation
'input_ns',
'work_ns',
'output_ns',
'idle_ns',
# end parameter estimation              
# experiment args
'input_bytes',
'output_bytes',
'work_ticks',
'sleep_ns',
# end experiment args            
'queue_ns',
'network_ns',
'delta_response_ns',
'upload_rate',
'download_rate',
'network_avg_rate',
'delta_input_ns',
'delta_output_ns',
'delta_idle_ns',
'delta_work_ns',
'delta_work_ticks',
'delta_procedure_ns',
'delta_last_arrival_ns',
'delta_last_departure',
'delta_app_last_arrival',
'delta_app_last_departure',
'app_last_arrival_func',
'app_last_departure_func',
'delta_func_last_arrival',
'delta_func_last_departure',
*[f'func_req_count_{i}' for i in range(29)],
*[f'func_arrival_fft_{i}' for i in range(7)],
*[f'app_arrival_fft_{i}' for i in range(7)],
'func_reqn_queue',
'sidecar_queue_len',
'func_avg_pod_count',
'app_avg_pod_count',
'func_last_pod_count',
'app_last_pod_count',
'func_last_starting_pod_count',
'app_last_starting_pod_count',
'func_last_terminating_pod_count',
'app_last_terminating_pod_count',
'func_avg_reqn_count',
'app_avg_reqn_count',
'func_last_reqn_count',
'app_last_reqn_count',
'func_pod_conc_target',
'func_pod_conc_max',
'func_avg_pod_conc',
'app_avg_pod_conc',
'func_last_pod_conc',
'app_avg_pod_conc',
*[f'thread_cpu_time_{i}' for i in CPU_TIME_STATS],
'thread_cpu_pc',
'proc_affinity_cpu_pc',
'proc_mem_pc',
'global_avg_cpu_pc',
'global_mem_available',
'global_proc_active',
'global_proc_idle',
'func_startup_dt',
'func_shutdown_dt']
print(f'len: {len(EVENT_FEAT)}')
EVENT_FEAT


SyntaxError: unterminated string literal (detected at line 36) (1424062711.py, line 36)

In [14]:
STATE = {
    "global":{
        'current_requests': 0,
        'last_arrival': None,
        'last_departure': None,
        'lock': threading.Lock()
    },
    "apps":dict(
        [*[app_id,
            {'current_requests': set(),
            'last_arrival': None,
            'last_departure': None,
            'lock': threading.Lock()}]] for app_id in DF["app_id"].unique()),
    "func":dict(
        [*[func_id,
            {'current_requests': set(),
            'last_arrival': None,
            'last_departure': None,
            'lock': threading.Lock()}]] for func_id in DF["func_id"].unique()),
}

In [15]:
STATE

{'global': {'current_requests': 0,
  'last_arrival': None,
  'last_departure': None,
  'lock': <unlocked _thread.lock object at 0x7fa277dc20c0>},
 'apps': {115: {'current_requests': set(),
   'last_arrival': None,
   'last_departure': None,
   'lock': <unlocked _thread.lock object at 0x7fa277e81cc0>},
  24: {'current_requests': set(),
   'last_arrival': None,
   'last_departure': None,
   'lock': <unlocked _thread.lock object at 0x7fa278301bc0>},
  102: {'current_requests': set(),
   'last_arrival': None,
   'last_departure': None,
   'lock': <unlocked _thread.lock object at 0x7fa277dc1c80>},
  1: {'current_requests': set(),
   'last_arrival': None,
   'last_departure': None,
   'lock': <unlocked _thread.lock object at 0x7fa277dc3e00>},
  81: {'current_requests': set(),
   'last_arrival': None,
   'last_departure': None,
   'lock': <unlocked _thread.lock object at 0x7fa277dc3d00>},
  98: {'current_requests': set(),
   'last_arrival': None,
   'last_departure': None,
   'lock': <unlocke

In [ ]:
class Event:
    def __init__(self, ts, app, func, req, duration, duration_pc = (.25,.25,.25,.25), arrival, ):
        self.abs_init_ts = time.perf_counter()
        self.app = app
        self.func = func
        self.request = rid
        self.duration = duration
        self.sched_arrival = arrival
        self.input_ns, self.busy_ns, self.output_ns, self.idle_ns = \
            map(lambda i: int(duration*duration_pc[i]), range(4))

        
        # aftermath
        self.abs_end_ts = None
        self.service_dur = None
        self.delta_service_dur = None
        
    def finish():
        self.abs_end_ts = time.perf_counter()
        self.service_dur = self.abs_end_ts - self.abs_end_ts
        self.delta_service_dur = self.service_dur - self.duration
    

In [20]:
time.perf_counter()

21370.078704436